In [16]:
import ase
import requests
import numpy as np
import json
from pymatgen import MPRester, Structure
from pymatgen.io.ase import AseAtomsAdaptor
from cStringIO import StringIO
from os import remove

In [17]:
def anonymize(structure):
    n_atoms = len(structure.positions)
    structure.set_atomic_numbers([1] * n_atoms)
    structure.set_chemical_symbols(['H'] * n_atoms)
    return structure

def scale(structure, per='site'):
    if per == 'site':
        n_atoms = len(structure.positions)
    elif per == 'cell':
        n_atoms = 1  # scaling volume to 1/cell is the same as having 1 atom
    else:
        return ExitCode(400, '{} is not a valid `per` for scaling'.format(per))
    cell_volume = np.dot(np.cross(structure.cell[0], structure.cell[1]), structure.cell[2])
    new_cell = structure.get_cell() / np.cbrt(cell_volume / n_atoms)
    structure.set_cell(new_cell)
    new_pos = structure.get_positions() / \
        np.linalg.norm(structure.get_cell(), axis=1) * \
        np.linalg.norm(new_cell, axis=1)
    structure.set_positions(new_pos)
    return structure

In [18]:
structures = {}
with MPRester('0WqdPfXxloze6T9N') as mpr:
    structures['diamond'] = mpr.get_structure_by_material_id("mp-66")
    structures['gaas'] = mpr.get_structure_by_material_id("mp-2534")
    structures['rocksalt'] = mpr.get_structure_by_material_id("mp-22862")
    structures['perovskite'] = mpr.get_structure_by_material_id("mp-5827")
    structures['zincblende'] = mpr.get_structure_by_material_id("mp-10695")
    structures['wurtzite'] = mpr.get_structure_by_material_id("mp-561258")
    structures['fcc'] = mpr.get_structure_by_material_id("mp-30")
    structures['big_fcc'] = mpr.get_structure_by_material_id("mp-12628")
    structures['bcc'] = mpr.get_structure_by_material_id("mp-13")
    structures['triclinic'] = mpr.get_structure_by_material_id("mp-9481")
atoms = {name: AseAtomsAdaptor.get_atoms(structure) for name, structure in structures.iteritems()}
spkitMax = {1: max([len(structure.sites) for structure in structures.values()])}

In [19]:
atoms = {name: scale(anonymize(atom)) for name, atom in atoms.iteritems()}

In [30]:
atom_jsons = {}
for name, atom in atoms.iteritems():
    atom.write(filename='temp.json', format='json')
    with open('temp.json', 'r') as f:
        atom_jsons[name] = json.load(f)
remove('temp.json')

In [41]:
soaps = {}
for name, atom_json in [('fcc', atoms['fcc'])]:# atom_jsons.iteritems():
    payload = {'atoms': atom_json,
               'spkitMax': json.dumps(spkitMax), 'nocenters': None,
               'gaussian_width': 0.5, 'cutoff': 3.5,
               'cutoff_transition_width': 0.5, 'nmax': 8,
               'lmax': 6}
    r = requests.get('http://127.0.0.1:5000/v1/fingerprints/soap/', params=payload)
    soaps[name] = r.json()

TooManyRedirects: Exceeded 30 redirects.